### 01 - Cleaning the datasets

This notebook outlines the cleaning process of multiple dataset involving book
sales and rating so it can be used for EDA and analysis later on.
Fortunately, most of the datasets are clearly label and preprocessed.
The main objectives would be replacing/removing missing values and joining
necessary tables.

The 4 datasets are as followed:

1. [publishers](https://corgis-edu.github.io/corgis/csv/publishers/)
    * Ebook sales data from Amazon for 27k titles in 2015
2. [BX-Book-Rating](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)
    * Rating info on over 270k titles
3. [BX-Books](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)
    * Books info on over 270k title above. Lacking isbn!
4. [goodreads](https://www.kaggle.com/jealousleopard/goodreadsbooks#books.csv)
    * Goodreads book dataset including rating and reviews

After cleaning, there will be 3 datasets with label for specific use:
1. book_sales: contains ebook sales data.
2. book_goodreads: contains books info pulled from good reads.
3. book_rating: contains book rating from multiple sources.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn-white')

## book_sales data

Steps:
1. Lower column name case and change to snake cases.
2. Since multiple sales metrics are recorded, keep only the
unit sales column to ensure linearity.
3. Adding price_range column for eda. Based on Amazon ebook price range
4 ranges are:cheap ()
       a) cheap: 0 - 2.99
       b) normal: 2.99 - 9.99
       c) high: 9.99 - 19.99
       d) extra: 19.99+

In [2]:
# Loading Publisher dataset
data_path = 'D:\\PycharmProjects\\springboard\\data\\'
sales = pd.read_csv(f'{data_path}publishers.csv')

# Replace dot and space in columns name. Remove the word statistic in column name
sales.columns = sales.columns.str.replace(r'[\.\s]', '_').str.replace('statistics_', '')

# Remove multiple revenues and gross sales columns as these will create Multicollinearity 
# We only want units_sold in this case
sales = sales.drop(sales.columns[2:6], axis=1)
sales = sales.drop('sales_rank', axis=1)

# Cut prices into range for further analysis
sales['price_range'] = pd.cut(sales.sale_price, bins=[0, 2.99,9.99,19.99, max(sales.sale_price)], 
                           labels=['cheap','normal','high','extra'])

# Save the cleaned data with new name
sales.to_csv(f'{data_path}book_sales.csv')

# Final look at the dataframe
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27027 entries, 0 to 27026
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   genre                     27027 non-null  object  
 1   sold_by                   27027 non-null  object  
 2   daily_average_units_sold  27027 non-null  int64   
 3   publisher_name            27027 non-null  object  
 4   publisher_type            27027 non-null  object  
 5   average_rating            27027 non-null  float64 
 6   sale_price                27027 non-null  float64 
 7   total_reviews             27027 non-null  int64   
 8   price_range               27027 non-null  category
dtypes: category(1), float64(2), int64(2), object(4)
memory usage: 1.7+ MB


## book_rating data

The cleaned dataset will combine BX-Book-Ratings and BX-Books. This is
a simple data with just the ratings and the number of ratings.

Notes: Before loading the data, there are some anomaly in the book titles that
are manually removed (',' conflicting with csv separator).

Steps:
1. Lowercase the columns and change them to snake case.
2. Group the rating by ISBN and calculate the mean. Create the count of
rating column afterward.
3. Joining two tables by ISBN.

In [3]:
# Load books data set and clean up column names. Omitted last 3 columns since they are
# links only
books = pd.read_csv(f'{data_path}BX-Books.csv', sep=';', error_bad_lines=True,
                    usecols=[0,1,2,3,4], encoding='ISO-8859-1', index_col='ISBN',
                    low_memory=False)
books.columns = books.columns.str.lower().str.replace('-','_')

# Count
books.count()

book_title             271379
book_author            271378
year_of_publication    271379
publisher              271377
dtype: int64

There are multiples reviews of the same book (isbn) from different users. Thus, we will
get the mean rating as the metric to merge into books reviews.

In [4]:
# Load reviews data. We also lower case and snake_case column names
reviews = pd.read_csv(f'{data_path}BX-Book-Ratings.csv', sep=';', error_bad_lines=True,
                      encoding='ISO-8859-1', usecols=[1,2])
reviews.columns = reviews.columns.str.lower().str.replace('-', '_')

# Group by isbn and get the number of rating
total_rating = reviews.groupby('isbn').count()

# Group by isbn and get mean rating
reviews = reviews.groupby('isbn').mean()
reviews['total_rating'] = total_rating

# print info on reviews
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 340556 entries,  0330299891 to Ô½crosoft
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   book_rating   340556 non-null  float64
 1   total_rating  340556 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 7.8+ MB


In [5]:
# Merge books and reviews on isbn. Leave reviews without the isbn
rating = pd.merge(books, reviews, how='left', left_index=True, right_index=True)
rating = rating.dropna()

# save for future use
rating.to_csv(f'{data_path}book_rating.csv')

# first look
rating.info()
rating.head()

# Clear out unused data frames
del books
del reviews

<class 'pandas.core.frame.DataFrame'>
Index: 270167 entries, 0195153448 to 0767409752
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   book_title           270167 non-null  object 
 1   book_author          270167 non-null  object 
 2   year_of_publication  270167 non-null  object 
 3   publisher            270167 non-null  object 
 4   book_rating          270167 non-null  float64
 5   total_rating         270167 non-null  float64
dtypes: float64(2), object(4)
memory usage: 14.4+ MB


## Goodread books data

Used for rating prediction

There are some problems with a few lines in the dataset that needs correction since they use
comma instead of dot-comma in the author name. However, there are less than 5 of these so manual 
fixing is efficient. 

In [7]:
# Load in the data
goodread_books = pd.read_csv(f'{data_path}goodread_books.csv')

# Remove bookID, isbn, isbn13, and publication_date
goodread_books = goodread_books.drop(['bookID', 'isbn','isbn13'], axis=1)

# remove space from column name
goodread_books.columns = goodread_books.columns.str.strip()

# Title length
goodread_books['title_length'] = goodread_books.title.str.len()

# Convert publishing date to datetime
goodread_books['publication_date'] = pd.to_datetime(goodread_books.publication_date, errors='coerce')

# This two book has a very bad publication date
print(goodread_books[goodread_books.publication_date.isnull()])

# Remove na 
goodread_books = goodread_books.dropna()

# info
goodread_books.info()

# Save for later
goodread_books.to_csv(f'{data_path}/book_goodread.csv')

                                                   title  \
8180   In Pursuit of the Proper Sinner (Inspector Lyn...   
11098         Montaillou  village occitan de 1294 à 1324   

                                               authors  average_rating  \
8180                                 Elizabeth  George            4.10   
11098  Emmanuel Le Roy Ladurie/Emmanuel Le Roy-Ladurie            3.96   

      language_code  num_pages  ratings_count  text_reviews_count  \
8180            eng        718          10608                 295   
11098           fre        640             15                   2   

      publication_date       publisher  title_length  
8180               NaT    Bantam Books            55  
11098              NaT  Folio histoire            42  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11125 entries, 0 to 11126
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0